**[In progress: The aim of this template is to provide structure for DS projects]**

# Project Brief

## Business Statement

## Requirements

## Limitations

# Data

## Extraction

In [ ]:
# importing flat files using NumPy
filename = 'name.txt'
data = np.loadtxt(filename, delimiter=',', skiprows=1, usecols=[0,2], dtype=str)

In [ ]:
# pickled files
import pickle 
with open('name.pkl', 'rb') as file:
    data = pickle.load(file)

print(data)

In [ ]:
# importing excel spreadsheets
file = 'name.xlsx'
data = pd.ExcelFile(file)

print(data.sheet_names)

df1 = data.parse('sheet_name') # sheet name, as a string
df2 = data.parse(0) # sheet index, as a float

In [ ]:
# importing csv files
data = pd.read_csv('name.csv')

"""use the below for multiple extractions
try:
except:
"""

In [ ]:
# importing SAS files
from sas7bdat import SAS7BDAT
with SAS7BDAT('name.sas7bdat') as file:
    df_sas = file.to_data_frame()

In [ ]:
# importing Stata files
data = pd.read_stata('name.dta')

In [ ]:
# importing HDF5 files
import h5py
filename = 'name.hdf5'
data = h5py.File(filename, 'r')

print(type(data))

# structure
for key in data.keys():
    print(key)

In [ ]:
# importing MATLAB files, keys = MATLAB variable names / values = objects assigned to variables
import scipy.io
filename = 'name.mat'
mat = scipy.io.loadmat(filename)

print(type(mat))

In [ ]:
# importing through SQL (sqlalchemy)

from sqlalchemy import create_engine
engine = create_engine('sqlite://name.sqlite')

table_names = engine.table_names()
print(table_names)

# querying
con = engine.connect()
rs - con.execute('SELECT * FROM table')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys() # setting the DF column names
con.close()

# through context manager
with engine.connect() as con:
    rs = con.execute('query')
    df = pd.DataFrame(rs.fetchmany(size=5)) # fetching specific number
    df.columns = rs.keys()

# querying directly via pandas
df = pd.read_sql_query('query', egine)

## Exploration & Processing

`Data type constraints`

In [ ]:
df.dtypes

In [ ]:
# feature classification

def classify_features(df):
    categorical_features = []
    non_categorical_features = []
    discrete_features = []
    continuous_features = []

    for column in df.columns:
        if df[column].dtype in ['object', 'bool', 'category']: 
            if df[column].nunique() < 15:
                categorical_features.append(column)
            else: 
                non_categorical_features.append(column)
        elif df[column].dtype in ['int64', 'float64']:
            if df[column].nunique() < 10:
                discrete_features.append(column)
            else: 
                continuous_features.append(column)
    return categorical_features, non_categorical_features, discrete_features, continuous_features


categorical, non_categorical, discrete, continuous = classify_features(df)

`Cleaning text data`

- Data inconsistency
- Fixed length violations
- Typos

In [ ]:
# checking for issues

df['col'].value_counts()

# using str.replace()

df['col'] = df['col'].str.replace('old element','new element')

# capitalize or lowercase 

df['col'] = df['col'].str.upper()
df['col'] = df['col'].str.lower()

# replacing elements with a condition

element_len = df['col'].str.len()
df.loc[element_len < number, 'col'] = np.nan # replace those falling under a specific condition with NaN

# asserting column does not contain specific, unwanted elements

assert df['col'].str.contains("element1|element2").any() == False # either element

# cleaning more complicated values: regular expressions

df['col'] = df['col'].str.replace(r'regex','replacement')
"""regex
  (?x) # Use free-spacing mode.
  <    # Match a literal '<'
  /?   # Optionally match a '/'
  \[   # Match a literal '['
  \d+  # Match one or more digits
  >    # Match a literal '>'
"""

`Handling duplicate values`

In [ ]:
# finding duplicate values

duplicates = df.duplicated() # by itself, it can lead to misleading results
df[duplicates]

In [ ]:
# treating not fully duplicate rows (e.g. all but one or two columns are the exact same)

# verification

duplicates = df.duplicated(subset=column_names, keep=False)
df[duplicates].sort_values(by='col1') # sorting to get a better picture of the duplicates

# dropping complete records only

df.drop_duplicates(inplace=True)

# group by column names and produce statistical summaries to combine rows

column_names = ['col1', 'col2', 'col3']
summaries = {'col1': 'max', 'col2': 'mean'} # create a dictionary in order to merge nearly identical rows
df = df.groupby(by=column_names).agg(summaries).reset_index()

`Handling missing values`

**Simple approaches**
- Drop missing data
- Impute with statistical measures *(mean, median, mode..)*

**More complex approaches**
- Imputing using an algorithmic approach
- Impute with machine learning models

**Visualization**
- missingno package: package for visualizing and understanding missing data

In [ ]:
import missingno as msno

# visualization
msno.matrix(df) # how missing values are distributed across columns
plt.show()

# isolate rows and validate
missing = df[df['col'].isna()] # pair with .describe()
complete = df[~df['col'].isna()] # pair with .describe()

# sort 

sorted_df = df.sort_values(by = 'col')
msno.matrix(sorted_df)
plt.show()

In [ ]:
# checking the proportion of missing values to the respective totals

df_nan = (df.isnull().sum() / len(df)) * 100
df_nan = df_na.drop(df_na[df_na == 0].index).sort_values(ascending=False)[:30]

missing_data = pd.DataFrame({'Missing Ratio' :df_na})
missing_data.head(10)

In [ ]:
# set a threshold to drop missing values (missing values as a % of total)

threshold = len(df) * 0.05

cols_to_drop = df.columns[df.isna().sum() <= threhold]

df.dropna(subset=cols_to_drop, inplace=True)

In [ ]:
# imputation 

cols_with_missing_values = df.columns[df.isna().sum() > 0]

for col in cols_with_missing_values[:-1]:
    df[col].fillna(df[col].mode()[0])

# option 2: simple imputation

col_mean = df['col'].mean()
df_imputed = df.fillna({'col': col_mean})

# imputation by sub-group

df_dict = df.groupby("col1")["col2"].median().to_dict()

df["col2"] = df["col2"].fillna(df["col1"].map(df_dict))

`Handling outliers`

In [ ]:
# identifying thresholds

# 75th percentile
seventy_fifth = df["col"].quantile(0.75)

# 25th percentile
twenty_fifth = df["col"].quantile(0.25)

# Interquartile range
df_iqr = seventy_fifth - twenty_fifth

In [ ]:
# identifying outliers

# Upper threshold
upper = seventy_fifth + (1.5 * df_iqr)

# Lower threshold
lower = twenty_fifth - (1.5 * df_iqr)

In [ ]:
# subsetting the data

df[(df["col"] < lower) | (df["col"] > upper)] \
      [["col1", "col2", "col3"]] # columns to display

In [ ]:
# dropping outliers

no_outliers = df[(df["col"] > lower) | (df["col"] < upper)]

print(no_outliers["col"].describe())

`Membership constraints`

- Dropping data
- Remapping categories
- Inferring categories

In [ ]:
# finding inconsistent categories
inconsistent_categories = set(df['col1']).difference(categories['col1']) # using set() gives us the unique values

# get rows with inconsistent categories
inconsistent_rows = df['col1'].isin(inconsistent_categories)
df[inconsistent_rows] # subset based on boolean values

# dropping inconsistent categories
consistent_data = df[~inconsistent_rows] # returns everything except inconsistent rows

In [ ]:
# collapsing data into categories

# creating category ranges and names
ranges = [0, integer, integer, np.inf]
group_names = ['label1','label2','label3']

# creating new group column
df['new_col'] = pd.cut(df['col1'], bins=ranges,
                    labels=group_names)

df[['new_col', 'col1']]

In [ ]:
# mapping categories to fewer ones

# mapping dictionary and replace
mapping = {'category1':'new_category1', 'category2':'new_category1', 'category3':'new_category1',
            'category4':'new_category2', 'category5':'new_category2'}

df['col'] = df['col'].replace(mapping)

# verification
df['col'].unique()

`Date range constraints`

 - Understand dataset before dropping values
 - Set custom minimums and maximums
 - Treat as missing and impute
 - Set custom values depending on business assumptions

In [ ]:
# where dates are in the future
import datetime as dt

# check data type to see if it needs conversion

df.dtypes
df['col'] = pd.to_datetime(df['col']).dt.date

today_date = dt.date.today()
df[df['col'] > dt.date.today()] # isolate data

# option 1: drop the rows

df = df[df['col'] < today_date]

# option 2: drop values using .drop()

df.drop(df[df['col'] > today_date].index, inplace = True)

# option 3: hardcode dates with upper limit

df.loc[df['col'] > today_date, 'col'] = today_date

# assertion

assert df.col.max().date() <= today_date # chaining with date() to get a date not a timestamp

`Uniformity`

datetime

In [ ]:
# convert to datetime

df['col'] = pd.to_datetime(df['col'], 
                            # attempt to infer format of each date
                            infer_datetime_format=True,
                            # return Na for rows where conversion failed
                            errors = 'coerce') # NaT

# conversion using dt.strftime()

df['col'] = df['col'].dt.strftime("%d-%m-%Y") 

"""treating ambiguous dates
Convert to NA and treat accordingly
Infer format by understanding data source
Infer format by understanding previous and subsequent data in DataFrame
"""


`Cross field validation`

- Use of multiple fields in the dataset to sanity check data integrity


In [ ]:
# sum of columns to total sanity check

sum_columns = df[['col1', 'col2', 'col3']].sum(axis = 1) # row-wise summing
filter = sum_columns == df['total_column']

# find and filter out rows with inconsistencies

inconsistent_data = df[~filter]
consistent_data = df[filter]

`Categorical encoding`

In [ ]:
# multi-column label encoding

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # column names to encode

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

`Normalization & Standardization`

## Insights

`Correlation`

In [ ]:
# checking the strength of relationship between variables

df.corr()

In [ ]:
# correlation heatmap

sns.heatmap(df.corr(), annot=True)
plt.show()

`Statistical analysis on scalar data`

In [ ]:
labels = ["category1", "category2", "category3", "category4"]

bins = [0, twenty_fifth, median, seventy_fifth, maximum] # min, median, quartiles and max to be assigned to labels/categories

pd.cut(data, # pass the data
      labels, # set the labels
      bins # provide the bins
      )

`Data summarization`

- Groupings

In [ ]:
# adding summary statistics to the dataframe

# standard deviation example using lambda
df["std_dev"] = df.groupby("col")["col1"].transform(lambda x: x.std()) # for each x, transform to the respective standard deviation

## Graphs

# Model Building

`Model Preparation`

In [ ]:
# verifying class imbalance after data splitting

classes = unique(target_train)
total = len(target_train)
for c in classes:
    n_examples = len(target_train[target_train==c])
    percent = n_examples / total * 100
    print('> Class = %d : %d/%d (%.1f%%)' % (c, n_examples, total, percent))

`Model Selection`

- Benchmark ('Dummy') model
- Classification v. Regression

## Analysis

`Model creation and hyperparameter tuning`

- GridsearchCV / RandomizedSearchCV
- Feature importance
- Metric selection
- Bayesan optimization
- Keras tuner

## Evaluation